# Levinson-Durbin algorithm

One wants to solve a linear equation of the form:

$$
\begin{bmatrix}
\gamma_x[0] & \gamma_x[1] & \cdots & \gamma_x[p-1] \\
\gamma_x[1] & \gamma_x[0] & \cdots & \gamma_x[p-2] \\
\gamma_x[2] & \gamma_x[1] & \cdots & \gamma_x[p-3] \\
\vdots & \vdots & \ddots & \vdots \\
\gamma_x[p-1] & \gamma_x[p-2] & \cdots & \gamma_x[0]
\end{bmatrix}
\begin{bmatrix}
a_1 \\
a_2 \\
\vdots \\
a_p \\
\end{bmatrix}
=
\begin{bmatrix}
\gamma_x[1] \\
\gamma_x[2] \\
\vdots \\
\gamma_x[p]\\
\end{bmatrix}
$$

To solve this system, one can employs the recursive **Levinson-Durbin algorithm** [1]. See [2] for a proof. It runs in $\mathcal{O}(p^2)$ time whereas vanilla Gaussian elimination runs in $\mathcal{O}(p^3)$, which is a drastic amelioration. The matrix of the system is said *Toeplitz*, it takes advantage of that. The diagonal has to be non-zero, but any non-null signal will check that condition.

Levinson-Durbin algorithm was coined to solve a linear equation. This linear equation, famously known as the *Yule-Walker* equation, is a direct consequence of the autoregressive model and definition of autocorrelation. In Time Series modeling, an autoregressive model of order $p$ (AR(p)) is an ansatz for the dynamic of $(x[n])_{n \in \mathbb{N}}$ of the form:
$$
x[n] = \sum\limits_{i=1}^p a_i x[n-i] = a_1 x[n-1] + a_2x[n-2] + \cdots a_p x[n-p].
$$
The question lies in the choice of the coefficients $(a_i)_{i \in \{1, \cdots, p\}}$. One assumes to have access to (at least an estimator of) the autocorrelation up until order $p$. It is classically defined as:
$$
\gamma_x[m] = \mathbb{E}[x \cdot \tau_m \bar x], \, \text{with} \, \tau_m x[n] = x[n-m], \, \text{the sliding operator}.
$$
The bar denotes the complex conjugates here.


[1] https://en.wikipedia.org/wiki/Levinson_recursion

[2] [6.341: Discrete-Time Signal Processing, MIT, Lecture 13: The Levinson-Durbin Recursion](https://ocw.mit.edu/courses/6-341-discrete-time-signal-processing-fall-2005/06e8ddb9555ede1b094f5dc9d17ea254_lec13.pdf)

In [ ]:
import numpy as np

def build_Yule_Walker(p: int, gamma: np.ndarray):
    """
    Build the parameters of the Yule-Walker equation.
    Args:
        p: order of the autoregressive model.
        gamma: vector of autocorrelation.
    Returns:
        A Toeplitz matrix of the p first coefficients and a vector composed the p last coefficients of gamma.
    """
    assert p+1 == len(gamma), f"Correlation vector is not the right shape for order {p}."
    if isinstance(gamma, list):
        gamma = np.array(gamma)
    gamma_matrix = gamma[None,:-1]
    
    gamma_tmp = gamma[:-1]
    for i in range(1 ,p):
        tmp = gamma_matrix[-1]
        tmp = np.delete(tmp, -1)
        tmp = np.append(gamma_tmp[i], tmp)
        gamma_matrix = np.append(gamma_matrix, tmp[None,:], axis=0)
    return gamma_matrix, gamma[1:]

def Levinson_Durbin(p: int, gamma: np.ndarray):
    """
    Apply Levinson-Durbin algorithm to solve Yule-Walker equation.
    
    """
    assert p + 1 == len(gamma), "The order of the model should match the vector size."
    if isinstance(gamma, list):
        gamma = np.array(gamma)
    k=1
    a = np.array([gamma[1]/gamma[0]])
    while k <= p-1:
        c = gamma[1:k+1]
        alpha = gamma[k+1] - np.dot(a, c[::-1])
        beta = gamma[0] - np.dot(a, c)

        coeff = alpha/beta
        a = a - coeff*a[::-1]
        a = np.append(a, coeff)
        k += 1
    return a

#Sanity check
if __name__=="__main__":
    gamma = [1, .5, .8, .2, 0]
    gamma_x, autocorrel = build_Yule_Walker(4, gamma)
    auto_reg_coeff = np.linalg.solve(gamma_x, autocorrel)
    Levinson_sol = Levinson_Durbin(4, gamma)
    print(np.linalg.norm(auto_reg_coeff - Levinson_sol))

5.259223848564703e-15
